<h1>빅데이터분석기사 작업형 2 문제 연습</h1>


<h3>데이터 로드</h3>

<h3>데이터 결측치, 특성 파악</h3>

In [78]:
import pandas as pd
import numpy as np

X_test_path = 'data/X_test.csv'
X_train_path = 'data/X_train.csv'
y_train_path = 'data/y_train.csv'
result_path = 'data/res.csv'

# 맥(utf-8)과 윈도우(cp949)의 인코딩 방식 차이로 cp949 인코딩을 이용하여 에러 방지
X_test = pd.read_csv(X_test_path, encoding='cp949')
X_train = pd.read_csv(X_train_path, encoding='cp949')
y_train = pd.read_csv(y_train_path, encoding='cp949')

print('X_train 레코드 수:', len(X_train))
print('y_train 레코드 수:', len(y_train))
print('X_test 레코드 수:', len(X_test))

print(X_train)
print(y_train)

# cust_id 컬럼 모두 제거
#X_train = X_train.iloc[:,1:]
#X_test = X_test.iloc[:,1:]
#y_train = y_train.iloc[:,1]



X_train 레코드 수: 3500
y_train 레코드 수: 3500
X_test 레코드 수: 2482
      cust_id       총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0           0   68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1           1    2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2           2    3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3           3   16077620   4935000        NaN      기타   광주점    18  2.444444   
4           4   29050000  24000000        NaN      보석  본  점     2  1.500000   
...       ...        ...       ...        ...     ...   ...   ...       ...   
3495     3495    3175200   3042900        NaN      골프  본  점     1  2.000000   
3496     3496   29628600   7200000  6049600.0    시티웨어  부산본점     8  1.625000   
3497     3497      75000     75000        NaN    주방용품   창원점     1  1.000000   
3498     3498    1875000   1000000        NaN     화장품  본  점     2  1.000000   
3499     3499  263101550  34632000  5973000.0      기타  본  점    38  2.421

<h3>Preprocessing</h3>

In [98]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

# Label Encoding

X_train.loc[:,['주구매상품','주구매지점']] \
= X_train.loc[:,['주구매상품','주구매지점']].apply(LabelEncoder().fit_transform)
#print(X_train.loc[:,['주구매상품','주구매지점']])

X_test.loc[:,['주구매상품','주구매지점']] \
= X_test.loc[:,['주구매상품','주구매지점']].apply(LabelEncoder().fit_transform)
#print(X_test.loc[:,['주구매상품','주구매지점']])


# drop null
X_train.loc[:,['환불금액']] = X_train.loc[:,'환불금액'].fillna(0)
X_test.loc[:,['환불금액']] = X_test.loc[:,['환불금액']].fillna(0)
    
print(X_train)

# Check Correlation
x_train = pd.merge(X_train, y_train, how = 'inner', on='cust_id')
x_train_corr = x_train.corr()
print(x_train_corr['gender'].abs())
# ['총구매액','최대구매액','환불금액','내점일수','주말방문비율']

x_train = x_train.loc[:,['총구매액','최대구매액','환불금액','내점일수','주말방문비율']]
y_train = y_train['gender']
print(x_train)

      cust_id       총구매액     최대구매액       환불금액  주구매상품  주구매지점  내점일수   내점당구매건수  \
0           0   68282840  11264000  6860000.0      5      0    19  3.894737   
1           1    2136000   2136000   300000.0     21     19     2  1.500000   
2           2    3197000   1639000        0.0      6      1     2  2.000000   
3           3   16077620   4935000        0.0      5      2    18  2.444444   
4           4   29050000  24000000        0.0     15      8     2  1.500000   
...       ...        ...       ...        ...    ...    ...   ...       ...   
3495     3495    3175200   3042900        0.0      3      8     1  2.000000   
3496     3496   29628600   7200000  6049600.0     22      9     8  1.625000   
3497     3497      75000     75000        0.0     32     21     1  1.000000   
3498     3498    1875000   1000000        0.0     41      8     2  1.000000   
3499     3499  263101550  34632000  5973000.0      5      8    38  2.421053   

        주말방문비율  구매주기  
0     0.527027    17  
1    

<h3>Modeling, Predict</h3>

In [109]:
from sklearn import linear_model

model = linear_model.LogisticRegression()
model.fit(X = x_train, y = y_train)

print('Logistic Regression model Score: ',model.score(x_train, y_train))

pred = model.predict_proba(X_test.loc[:,['총구매액','최대구매액','환불금액','내점일수','주말방문비율']])
res = pd.DataFrame(np.c_[X_test['cust_id'],pred],columns=['cust_id','gender'])

res.to_csv(result_path,index=False)

Logistic Regression model Score:  0.624


ValueError: Shape of passed values is (2482, 3), indices imply (2482, 2)